In [223]:
import os 
from pathlib import Path

import pandas as pd
import numpy as np


In [224]:
from sbscorer.sblegos.TransactionAnalyser import TransactionAnalyser as txa

In [225]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)
PATH_TO_GR18 = os.path.join(current_dir.parent.parent, FOLDER_NAME)
voter_tx_file_name = 'gg18_votes.csv'
passport_file_name = 'gg18_voter_passports.csv'

tx_file_name = 'gr_18_optimism_all_tx.csv'

In [226]:
votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))
passports = pd.read_csv(os.path.join(PATH_TO_GR18, passport_file_name))

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_1516\1560736169.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))


In [227]:
round_names = votes['round_name'].unique()
round_names

array(['Climate Round', 'Web3 Open Source Software',
       'Web3 Community and Education ',
       'Global Chinese Community for Public Goods - GR18', 'Web3 Social',
       'Zuzalu Continuous Innovation', 'Meta Pool LatAm GG18',
       'ReFi DAO Local Node ', 'Token Engineering',
       'Ethereum Infrastructure', 'Arbitrum Domain Round'], dtype=object)

In [228]:
# Load tx data
df_tx = pd.read_csv(os.path.join(PATH_TO_EXPORT, tx_file_name))
df_tx.head()

,tx_hash,block_timestamp,from_address,to_address,gas_limit,gas_used,tx_fee,eth_value,__row_index
0,0x863c5b543d29b329891fdec341845a73c2525e6dc212...,2023-08-24T20:22:03.000Z,0xf4af0314a06ea7376da5bdecc33f6f9800f1ae28,0x7f5c764cbc14f9669b88837ca1490cca17c31607,53067,53067,0.000029,0.000000,0
1,0xfdffb6eb77e72ca9f54297ebcc7644ea55db521f64c7...,2023-08-24T22:05:41.000Z,0x87dbffd0ed90b2252856faf753437a7bc9fcfa7b,0xb009b5707e6e0dc9830a1262de37f055972e6480,219895,139765,0.000058,0.000777,1
2,0x7b416f625fd29c2da17a7ad35b1c6dc7e0cf5b548249...,2023-08-24T16:10:17.000Z,0x532c1fc2940ec38d5c5e7fc8b8aa4aaddcaa17d3,0x46ce3b08bdef4461fb444d3746fd97f0af11dccf,731592,467423,0.000093,0.000000,2
3,0xbed4ce09f5f5c4d7aac7e310dfce11d94bf86f7da6d5...,2023-08-24T16:09:33.000Z,0x0596c217a79ea00e5dafc43778138ea61fd67ee2,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119625,119625,0.000087,0.000000,3
4,0xb60502458c8de31606b84e0aed102ac83629726d8734...,2023-08-24T22:08:15.000Z,0xc1991c8bda29507991eeb230ff2063c2ea74a34c,0xa062ae8a9c5e11aaa026fc2670b0d65ccc8b2858,246899,229592,0.000070,0.000000,4


In [229]:
df_tx.drop(columns=['__row_index'], inplace=True)

In [230]:
# Choose round name to compute features
round_name = round_names[0]

In [231]:
CONTRACT_CREATION_AD_NAME = '0x_contract_creation'

In [232]:
df_tx.loc[df_tx['to_address'].isna(), 'to_address'] = CONTRACT_CREATION_AD_NAME

In [233]:
df_tx.isna().sum()

tx_hash            0
block_timestamp    0
from_address       0
to_address         0
gas_limit          0
gas_used           0
tx_fee             0
eth_value          0
dtype: int64

In [234]:
df_tx.shape

(6894506, 8)

In [235]:
votes_round = votes.loc[votes['round_name'] == round_name, :].copy()

In [236]:
round_voters = votes_round['voter'].unique()
print(f'Number of voters in round {round_name}: {len(round_voters)}')

Number of voters in round Climate Round: 5610


In [237]:
df_tx_EOA_FROM = df_tx.copy()
df_tx_EOA_FROM['EOA'] = df_tx_EOA_FROM['from_address']

In [238]:
df_tx_EOA_TO = df_tx.copy()
df_tx_EOA_TO['EOA'] = df_tx_EOA_TO['to_address']

In [239]:
round_voters

array(['0x7c52D1870423A266f6b0626fBC20EcA148D0300B',
       '0x8073639B11994C549eDa58fC3cd7132a72aaDF10',
       '0xCe462C13F461DEEB3Ec2dAaDd5e7d5F90Bb8F0b2', ...,
       '0x0e746F622F5211eA6b3798f12C18897968323EE2',
       '0xc37e705f34801896121B2Da24327332Cb7DC2CA7',
       '0xCd20986CB96B6a602427379F62deFd5Adef6153B'], dtype=object)

In [240]:
df_tx_eoa = pd.concat([df_tx_EOA_FROM, df_tx_EOA_TO])

In [241]:
df_tx_eoa.shape

(13789012, 9)

In [242]:
df_tx_eoa = df_tx_eoa.loc[df_tx_eoa['EOA'] != CONTRACT_CREATION_AD_NAME, :]

In [243]:
df_tx_eoa.shape

(13787932, 9)

In [244]:
lowercase_converter = np.vectorize(str.lower)
round_voters = lowercase_converter(round_voters)

In [245]:
tx_analyser = txa(df_tx_eoa, round_voters)

In [251]:
len(tx_analyser.df_transactions)

892332

In [247]:
len(tx_analyser.array_address)

5610

In [248]:
len(round_voters)

5610

In [249]:
diff = np.setdiff1d(round_voters, tx_analyser.array_address)
print(len(diff))
diff

0


array([], dtype='<U42')

In [250]:
brea

NameError: name 'brea' is not defined

In [ ]:
df_features = tx_analyser.get_df_features()

In [ ]:
df_features.shape

(610, 19)

In [ ]:
df_features

,EOA,count_tx,less_10_tx,count_same_seed,count_same_seed_naive,same_seed,same_seed_naive,seed_suspicious,count_interact_other_ctbt,first_in_tx_from,first_in_tx_gas_limit,first_in_tx_gas_used,first_in_tx_eth_value,first_in_tx_timestamp,to_address,first_out_tx_gas_limit,first_out_tx_gas_used,first_out_tx_eth_value,first_out_tx_timestamp
0,0x00117fd41eab0f3d0c7d048a402c2e14d539473d,4,True,231,0,True,False,True,0,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,21000.0,21000.0,0.004370,2023-08-29T08:28:07.000Z,0x2297aebd383787a160dd0d9f71508148769342e3,234905,224891,0.003118,2023-05-28T02:17:27.000Z
1,0x0015f1d02b55ec9edd7536ca628fd30361252ebf,41,False,31,27,True,True,False,1,0x2d2cc0eb095e43204e0c087e07dbf95909650939,150000.0,21000.0,0.025000,2023-01-14T11:08:22.000Z,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,287571,215719,0.014335,2023-01-14T13:25:56.000Z
2,0x0035ab0a6508aa9fa7d093cd53d0fcbae1b6e926,7,True,1,1,True,True,False,1,0x4acd61b27509ac5afa151aedb9d073bce095dd7a,21000.0,21000.0,0.001500,2023-06-14T14:25:37.000Z,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119654,119654,0.000000,2023-06-14T14:26:25.000Z
3,0x003b26e03e7d5822fa24aa75ef778fddec8b8e4f,5,True,59,0,True,False,True,0,0x80c67432656d59144ceff962e8faf8926599bcf8,100000.0,21000.0,0.020170,2023-07-09T22:33:25.000Z,0xb49c4e680174e331cb0a7ff3ab58afc9738d5f8b,485526,419296,0.178721,2023-06-28T10:06:49.000Z
4,0x00d5cbf97a6e9786a1a270c4b4404c81d59d643b,8,True,0,0,False,False,False,0,NaN,NaN,NaN,NaN,NaN,0xe6266a11f1c10d6ccdcf96b31508c3dc3f541890,156036,149086,0.000690,2023-07-03T19:53:19.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,0xfdc37b75a4cd7cd48d52466e5744b935ea940cec,5,True,231,0,True,False,True,0,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,21000.0,21000.0,0.005321,2023-08-29T08:23:43.000Z,0x2297aebd383787a160dd0d9f71508148769342e3,234953,224939,0.001225,2023-05-27T17:46:39.000Z
606,0xfe5bdb59db8745f0dd23e532f2bd2935d996cce8,45,False,31,27,True,True,False,0,0x2d2cc0eb095e43204e0c087e07dbf95909650939,150000.0,21000.0,0.025000,2023-01-14T11:05:22.000Z,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,269948,199997,0.013199,2023-01-14T13:25:11.000Z
607,0xfef5a1a2b3754a2f53161eaaacb3eb889f004d4a,7,True,6,5,True,True,False,1,0x302e2a0d4291ac14aa1160504ca45a0a1f2e7a5c,21000.0,21000.0,0.002980,2023-08-21T03:03:03.000Z,0x2871742b184633f8dc8546c6301cbc209945033e,78323,71662,0.000595,2023-08-21T03:03:55.000Z
608,0xff350e4dbc227b236ada08c8e4bb39e639d2aa8e,6,True,59,0,True,False,True,0,0x80c67432656d59144ceff962e8faf8926599bcf8,100000.0,21000.0,0.018480,2023-07-09T22:33:39.000Z,0x2297aebd383787a160dd0d9f71508148769342e3,234953,224939,0.001225,2023-05-28T01:33:50.000Z


In [ ]:
df_features.to_csv(os.path.join(PATH_TO_EXPORT, f'features_{round_name}.csv'), index=False)